# Sensitivity Analysis Simulation

This notebook implements sensitivity analysis to demonstrate algorithm robustness across parameter variations.

## Analysis Scope
1. **Priority EV Ratio**: 10%, 15%, 20%, 25%, 30%
2. **Charger-to-EV Ratio**: 1:2 and 1:3
3. **Charging Window**: Tight (1hr), Medium (2hr), Relaxed (4hr)

## Output
- Table: Sensitivity analysis results
- Figure: Priority fulfillment vs parameter variations
- Figure: Cost vs parameter variations

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

# Set publication-quality plotting style
plt.rcParams.update({
    'font.size': 10,
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],
    'axes.linewidth': 0.8,
    'xtick.major.width': 0.8,
    'ytick.major.width': 0.8,
    'xtick.minor.width': 0.5,
    'ytick.minor.width': 0.5,
    'lines.linewidth': 1.2,
    'patch.linewidth': 0.8,
    'grid.linewidth': 0.5,
    'legend.frameon': True,
    'legend.fancybox': False,
    'legend.edgecolor': 'black',
    'legend.framealpha': 1.0,
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
    'savefig.pad_inches': 0.05
})

print('Imports complete. Ready for sensitivity analysis.')

## 1. Configuration Parameters

In [ ]:
# =============================================================================
# SENSITIVITY ANALYSIS CONFIGURATION
# =============================================================================

# Base simulation parameters
PERIOD = 5  # minutes
VOLTAGE = 220  # volts
DEFAULT_BATTERY_POWER = 21  # kW max charging rate
BATTERY_CAPACITY_RANGE = (60, 80)  # kWh
RANDOM_SEED = 42

# Sensitivity Analysis Parameters
PRIORITY_EV_RATIOS = [0.10, 0.15, 0.20, 0.25, 0.30]  # 10% to 30% in 5% increments
CHARGER_TO_EV_RATIOS = ['1:2', '1:3']  # Two ratio configurations

# Charging window configurations (in hours)
CHARGING_WINDOWS = {
    'tight': 1.0,      # 1 hour window
    'medium': 2.0,     # 2 hour window  
    'relaxed': 4.0     # 4 hour window
}

# Network configurations based on charger-to-EV ratio
# For 15 EVSEs: 1:2 = 30 sessions, 1:3 = 45 sessions
NETWORK_CONFIGS = {
    '1:2': {'n_evse': 15, 'n_sessions': 30},
    '1:3': {'n_evse': 15, 'n_sessions': 45}
}

# Algorithm names for comparison
ALGORITHMS = ['MPC-AQ', 'RR', 'LLF', 'Uncontrolled']

# Color palette - eye-friendly, high contrast for publication
COLORS = {
    'MPC-AQ': '#2E86AB',      # Blue
    'RR': '#A23B72',          # Magenta
    'LLF': '#F18F01',         # Orange
    'Uncontrolled': '#C73E1D' # Red
}

print('Configuration loaded:')
print(f'  Priority EV ratios: {[f"{r*100:.0f}%" for r in PRIORITY_EV_RATIOS]}')
print(f'  Charger-to-EV ratios: {CHARGER_TO_EV_RATIOS}')
print(f'  Charging windows: {list(CHARGING_WINDOWS.keys())}')

## 2. Session Data Generator for Sensitivity Analysis

In [ ]:
class SensitivitySessionGenerator:
    """
    Generates EV charging session data for sensitivity analysis.
    
    Creates sessions with configurable:
    - Priority EV ratio
    - Number of sessions (based on charger-to-EV ratio)
    - Charging window duration
    """
    
    def __init__(self, n_evse=15, period=5, battery_capacity_range=(60, 80),
                 max_charging_power=21, seed=42):
        self.n_evse = n_evse
        self.period = period
        self.battery_capacity_range = battery_capacity_range
        self.max_charging_power = max_charging_power
        self.seed = seed
        
        # Operating hours: 6 AM to 7 PM (13 hours)
        self.operating_start = 6  # 6 AM
        self.operating_end = 19   # 7 PM
        
    def generate_sessions(self, n_sessions, priority_ratio, 
                         charging_window_hours=2.0, seed=None):
        """
        Generate session data for sensitivity analysis.
        
        Parameters:
        -----------
        n_sessions : int
            Number of EV sessions to generate
        priority_ratio : float
            Fraction of EVs that are priority (0.0 to 1.0)
        charging_window_hours : float
            Average charging window duration in hours
        seed : int, optional
            Random seed for reproducibility
            
        Returns:
        --------
        list of dict: Session data compatible with ACN-Sim
        """
        if seed is None:
            seed = self.seed
        np.random.seed(seed)
        
        sessions = []
        n_priority = int(n_sessions * priority_ratio)
        priority_indices = set(np.random.choice(n_sessions, n_priority, replace=False))
        
        for i in range(n_sessions):
            # Generate arrival time (uniform across operating hours)
            # Leave room for departure before end of day
            max_arrival = self.operating_end - charging_window_hours
            arrival_hour = np.random.uniform(self.operating_start, max_arrival)
            
            # Generate departure based on charging window
            # Add some variance (+/- 30 minutes)
            window_variance = np.random.uniform(-0.5, 0.5)
            actual_window = max(0.5, charging_window_hours + window_variance)
            departure_hour = min(arrival_hour + actual_window, self.operating_end)
            
            # Ensure minimum 30-minute window
            if departure_hour - arrival_hour < 0.5:
                departure_hour = arrival_hour + 0.5
            
            # Generate energy requirement
            battery_capacity = np.random.uniform(*self.battery_capacity_range)
            # SoC between 20-60% at arrival
            initial_soc = np.random.uniform(0.20, 0.60)
            energy_requested = battery_capacity * (1.0 - initial_soc)
            
            # Check if EV is priority
            is_priority = i in priority_indices
            
            # Convert to timesteps (periods)
            arrival_timestep = int(arrival_hour * 60 / self.period)
            departure_timestep = int(departure_hour * 60 / self.period)
            
            session = {
                'session_id': f'EV_{i:03d}',
                'arrival_hour': arrival_hour,
                'departure_hour': departure_hour,
                'arrival_timestep': arrival_timestep,
                'departure_timestep': departure_timestep,
                'energy_requested_kwh': energy_requested,
                'battery_capacity_kwh': battery_capacity,
                'max_charging_power_kw': self.max_charging_power,
                'is_priority': is_priority,
                'station_id': f'EVSE_{i % self.n_evse:02d}'
            }
            sessions.append(session)
        
        return sessions
    
    def sessions_to_dataframe(self, sessions):
        """Convert sessions list to pandas DataFrame for analysis."""
        return pd.DataFrame(sessions)

# Initialize generator
session_gen = SensitivitySessionGenerator(
    n_evse=15,
    period=PERIOD,
    battery_capacity_range=BATTERY_CAPACITY_RANGE,
    max_charging_power=DEFAULT_BATTERY_POWER,
    seed=RANDOM_SEED
)

print('Session generator initialized.')

## 3. Run Sensitivity Analysis Simulations

**Note:** This section provides the structure for running simulations. 
You'll need to integrate with your existing ACN-Sim simulation loop.

In [ ]:
# =============================================================================
# PLACEHOLDER: SIMULATION RESULTS
# Replace this with actual simulation runs using your ACN-Sim setup
# =============================================================================

def run_sensitivity_simulations():
    """
    Run sensitivity analysis simulations.
    
    This function should be integrated with your existing simulation code.
    Replace the placeholder results with actual simulation outputs.
    """
    results = []
    
    for ratio_config in CHARGER_TO_EV_RATIOS:
        config = NETWORK_CONFIGS[ratio_config]
        n_sessions = config['n_sessions']
        
        for priority_ratio in PRIORITY_EV_RATIOS:
            for window_name, window_hours in CHARGING_WINDOWS.items():
                # Generate sessions for this configuration
                sessions = session_gen.generate_sessions(
                    n_sessions=n_sessions,
                    priority_ratio=priority_ratio,
                    charging_window_hours=window_hours
                )
                
                # TODO: Replace with actual simulation
                # sim_result = run_acn_simulation(sessions, algorithm)
                
                # Placeholder results - replace with actual values
                for alg in ALGORITHMS:
                    result = {
                        'charger_ratio': ratio_config,
                        'n_sessions': n_sessions,
                        'priority_ratio': priority_ratio,
                        'priority_ratio_pct': f'{priority_ratio*100:.0f}%',
                        'charging_window': window_name,
                        'window_hours': window_hours,
                        'algorithm': alg,
                        # Placeholder metrics - replace with actual simulation results
                        'priority_fulfillment_pct': np.nan,
                        'total_energy_delivered_kwh': np.nan,
                        'total_cost_aud': np.nan,
                        'avg_fulfillment_pct': np.nan
                    }
                    results.append(result)
    
    return pd.DataFrame(results)

# Generate placeholder results structure
# sensitivity_results = run_sensitivity_simulations()
# print(f'Generated {len(sensitivity_results)} result configurations')

In [ ]:
# =============================================================================
# SAMPLE DATA FOR PLOTTING DEMONSTRATION
# Replace this with your actual simulation results
# =============================================================================

# Create sample data based on expected patterns from your research
# MPC-AQ should show ~100% priority fulfillment, others lower

np.random.seed(42)

sample_results = []

for ratio_config in CHARGER_TO_EV_RATIOS:
    n_sessions = NETWORK_CONFIGS[ratio_config]['n_sessions']
    
    for priority_ratio in PRIORITY_EV_RATIOS:
        for window_name, window_hours in CHARGING_WINDOWS.items():
            # Base performance decreases with higher priority ratio for non-MPC
            base_decline = (priority_ratio - 0.10) * 50  # More decline as priority increases
            
            # Window effect: tighter windows reduce performance
            window_effect = {'tight': -5, 'medium': 0, 'relaxed': 3}
            
            for alg in ALGORITHMS:
                if alg == 'MPC-AQ':
                    # MPC-AQ maintains high priority fulfillment
                    priority_fulfill = 99.5 + np.random.uniform(-0.5, 0.5)
                    cost = 33 + priority_ratio * 15 + np.random.uniform(-1, 1)
                elif alg == 'Uncontrolled':
                    priority_fulfill = 100  # Uncontrolled serves everyone but wastefully
                    cost = 38 + np.random.uniform(-1, 1)
                else:  # RR, LLF
                    priority_fulfill = 100 - base_decline + window_effect[window_name]
                    priority_fulfill = max(80, min(100, priority_fulfill + np.random.uniform(-2, 2)))
                    cost = 32 + np.random.uniform(-1, 2)
                
                result = {
                    'charger_ratio': ratio_config,
                    'n_sessions': n_sessions,
                    'priority_ratio': priority_ratio,
                    'priority_ratio_pct': f'{priority_ratio*100:.0f}%',
                    'charging_window': window_name,
                    'window_hours': window_hours,
                    'algorithm': alg,
                    'priority_fulfillment_pct': round(priority_fulfill, 2),
                    'total_energy_delivered_kwh': round(400 + np.random.uniform(-20, 20), 1),
                    'total_cost_aud': round(cost, 2),
                    'avg_fulfillment_pct': round(priority_fulfill - 5 + np.random.uniform(-3, 3), 2)
                }
                sample_results.append(result)

sensitivity_results = pd.DataFrame(sample_results)
print(f'Generated {len(sensitivity_results)} result configurations')
sensitivity_results.head(10)

## 4. Visualization: Priority Ratio Sensitivity

In [ ]:
def plot_priority_ratio_sensitivity(df, charger_ratio='1:3', window='medium',
                                    save_path=None):
    """
    Plot priority fulfillment and cost vs priority EV ratio.
    
    Parameters:
    -----------
    df : DataFrame
        Sensitivity analysis results
    charger_ratio : str
        Charger-to-EV ratio to filter ('1:2' or '1:3')
    window : str
        Charging window to filter ('tight', 'medium', 'relaxed')
    save_path : str, optional
        Path to save the figure
    """
    # Filter data
    mask = (df['charger_ratio'] == charger_ratio) & (df['charging_window'] == window)
    plot_data = df[mask].copy()
    
    # Create figure with two subplots
    fig, axes = plt.subplots(2, 1, figsize=(4.5, 5), sharex=True)
    
    # Define markers and linestyles
    markers = {'MPC-AQ': 'o', 'RR': 's', 'LLF': '^', 'Uncontrolled': 'D'}
    linestyles = {'MPC-AQ': '-', 'RR': '--', 'LLF': '-.', 'Uncontrolled': ':'}
    
    # --- Plot 1: Priority Fulfillment ---
    for alg in ALGORITHMS:
        alg_data = plot_data[plot_data['algorithm'] == alg]
        axes[0].plot(alg_data['priority_ratio'] * 100, 
                    alg_data['priority_fulfillment_pct'],
                    marker=markers[alg], 
                    linestyle=linestyles[alg],
                    color=COLORS[alg],
                    label=alg,
                    markersize=6,
                    linewidth=1.2)
    
    axes[0].set_ylabel('Priority EV Fulfillment (%)', fontsize=9)
    axes[0].set_ylim(75, 102)
    axes[0].legend(loc='lower left', fontsize=8, ncol=2)
    axes[0].grid(True, alpha=0.3, linewidth=0.5)
    axes[0].set_title(f'Sensitivity Analysis (Ratio: {charger_ratio}, Window: {window.capitalize()})',
                     fontsize=10, fontweight='bold')
    
    # --- Plot 2: Charging Cost ---
    for alg in ALGORITHMS:
        alg_data = plot_data[plot_data['algorithm'] == alg]
        axes[1].plot(alg_data['priority_ratio'] * 100, 
                    alg_data['total_cost_aud'],
                    marker=markers[alg], 
                    linestyle=linestyles[alg],
                    color=COLORS[alg],
                    label=alg,
                    markersize=6,
                    linewidth=1.2)
    
    axes[1].set_xlabel('Priority EV Ratio (%)', fontsize=9)
    axes[1].set_ylabel('Total Charging Cost (AUD)', fontsize=9)
    axes[1].grid(True, alpha=0.3, linewidth=0.5)
    
    # Set x-ticks
    axes[1].set_xticks([10, 15, 20, 25, 30])
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight', 
                   facecolor='white', edgecolor='none')
        print(f'Figure saved to: {save_path}')
    
    plt.show()
    
# Generate the plot
plot_priority_ratio_sensitivity(
    sensitivity_results, 
    charger_ratio='1:3', 
    window='medium',
    save_path='sensitivity_priority_ratio.png'
)

## 5. Visualization: Charging Window Sensitivity

In [ ]:
def plot_charging_window_sensitivity(df, charger_ratio='1:3', priority_ratio=0.25,
                                     save_path=None):
    """
    Plot priority fulfillment across different charging window durations.
    
    Parameters:
    -----------
    df : DataFrame
        Sensitivity analysis results
    charger_ratio : str
        Charger-to-EV ratio to filter
    priority_ratio : float
        Priority EV ratio to filter
    save_path : str, optional
        Path to save the figure
    """
    # Filter data
    mask = (df['charger_ratio'] == charger_ratio) & \
           (np.abs(df['priority_ratio'] - priority_ratio) < 0.01)
    plot_data = df[mask].copy()
    
    # Create figure
    fig, ax = plt.subplots(figsize=(4.5, 3.5))
    
    # Prepare data for grouped bar chart
    window_order = ['tight', 'medium', 'relaxed']
    x = np.arange(len(window_order))
    width = 0.2
    
    for i, alg in enumerate(ALGORITHMS):
        alg_data = plot_data[plot_data['algorithm'] == alg]
        # Sort by window order
        values = []
        for w in window_order:
            val = alg_data[alg_data['charging_window'] == w]['priority_fulfillment_pct'].values
            values.append(val[0] if len(val) > 0 else 0)
        
        offset = (i - 1.5) * width
        bars = ax.bar(x + offset, values, width, label=alg, color=COLORS[alg],
                     edgecolor='black', linewidth=0.5)
    
    ax.set_xlabel('Charging Window Duration', fontsize=9)
    ax.set_ylabel('Priority EV Fulfillment (%)', fontsize=9)
    ax.set_title(f'Window Sensitivity ({priority_ratio*100:.0f}% Priority EVs)',
                fontsize=10, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(['Tight\n(1 hr)', 'Medium\n(2 hr)', 'Relaxed\n(4 hr)'])
    ax.set_ylim(75, 105)
    ax.legend(loc='lower right', fontsize=8, ncol=2)
    ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight',
                   facecolor='white', edgecolor='none')
        print(f'Figure saved to: {save_path}')
    
    plt.show()

# Generate the plot
plot_charging_window_sensitivity(
    sensitivity_results,
    charger_ratio='1:3',
    priority_ratio=0.25,
    save_path='sensitivity_charging_window.png'
)

## 6. Visualization: Charger-to-EV Ratio Comparison

In [ ]:
def plot_charger_ratio_comparison(df, window='medium', save_path=None):
    """
    Compare performance across different charger-to-EV ratios.
    
    Parameters:
    -----------
    df : DataFrame
        Sensitivity analysis results
    window : str
        Charging window to filter
    save_path : str, optional
        Path to save the figure
    """
    # Filter data
    plot_data = df[df['charging_window'] == window].copy()
    
    # Create figure with two panels
    fig, axes = plt.subplots(1, 2, figsize=(7, 3.5), sharey=True)
    
    markers = {'MPC-AQ': 'o', 'RR': 's', 'LLF': '^', 'Uncontrolled': 'D'}
    linestyles = {'MPC-AQ': '-', 'RR': '--', 'LLF': '-.', 'Uncontrolled': ':'}
    
    for idx, ratio in enumerate(CHARGER_TO_EV_RATIOS):
        ratio_data = plot_data[plot_data['charger_ratio'] == ratio]
        
        for alg in ALGORITHMS:
            alg_data = ratio_data[ratio_data['algorithm'] == alg]
            axes[idx].plot(alg_data['priority_ratio'] * 100,
                          alg_data['priority_fulfillment_pct'],
                          marker=markers[alg],
                          linestyle=linestyles[alg],
                          color=COLORS[alg],
                          label=alg if idx == 0 else '',
                          markersize=5,
                          linewidth=1.2)
        
        axes[idx].set_xlabel('Priority EV Ratio (%)', fontsize=9)
        axes[idx].set_title(f'Charger Ratio {ratio}', fontsize=10, fontweight='bold')
        axes[idx].grid(True, alpha=0.3, linewidth=0.5)
        axes[idx].set_xticks([10, 15, 20, 25, 30])
        axes[idx].set_ylim(75, 102)
    
    axes[0].set_ylabel('Priority EV Fulfillment (%)', fontsize=9)
    axes[0].legend(loc='lower left', fontsize=7, ncol=2)
    
    plt.suptitle(f'Impact of Charger-to-EV Ratio ({window.capitalize()} Window)',
                fontsize=11, fontweight='bold', y=1.02)
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight',
                   facecolor='white', edgecolor='none')
        print(f'Figure saved to: {save_path}')
    
    plt.show()

# Generate the plot
plot_charger_ratio_comparison(
    sensitivity_results,
    window='medium',
    save_path='sensitivity_charger_ratio.png'
)

## 7. Summary Table Generation

In [ ]:
def generate_sensitivity_summary_table(df, save_path=None):
    """
    Generate a summary table for the paper.
    """
    # Pivot table: Priority ratio vs Algorithm for medium window, 1:3 ratio
    mask = (df['charger_ratio'] == '1:3') & (df['charging_window'] == 'medium')
    summary_data = df[mask].pivot_table(
        index='priority_ratio_pct',
        columns='algorithm',
        values='priority_fulfillment_pct',
        aggfunc='mean'
    )
    
    # Reorder columns
    summary_data = summary_data[ALGORITHMS]
    
    print('\n' + '='*60)
    print('Table: Priority EV Fulfillment (%) by Algorithm and Priority Ratio')
    print('Configuration: 1:3 Charger Ratio, Medium Window (2 hr)')
    print('='*60)
    print(summary_data.round(1).to_string())
    print('='*60)
    
    if save_path:
        summary_data.round(1).to_csv(save_path)
        print(f'Table saved to: {save_path}')
    
    return summary_data

# Generate summary table
summary_table = generate_sensitivity_summary_table(
    sensitivity_results,
    save_path='sensitivity_summary_table.csv'
)

## 8. Export All Results

In [ ]:
# Save complete results to Excel
output_file = 'sensitivity_analysis_results.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # Full results
    sensitivity_results.to_excel(writer, sheet_name='All_Results', index=False)
    
    # Summary by priority ratio
    summary_by_priority = sensitivity_results.groupby(
        ['priority_ratio_pct', 'algorithm']
    ).agg({
        'priority_fulfillment_pct': 'mean',
        'total_cost_aud': 'mean'
    }).round(2).reset_index()
    summary_by_priority.to_excel(writer, sheet_name='Summary_Priority', index=False)
    
    # Summary by charging window
    summary_by_window = sensitivity_results.groupby(
        ['charging_window', 'algorithm']
    ).agg({
        'priority_fulfillment_pct': 'mean',
        'total_cost_aud': 'mean'
    }).round(2).reset_index()
    summary_by_window.to_excel(writer, sheet_name='Summary_Window', index=False)

print(f'Results exported to: {output_file}')

## 9. Integration Instructions

To integrate this sensitivity analysis with your existing ACN-Sim simulation:

1. **Replace the sample data generation** in Section 3 with actual simulation calls
2. **Use the `SensitivitySessionGenerator`** to create sessions for each configuration
3. **Run your existing simulation loop** (MPC, RR, LLF, Uncontrolled) for each session set
4. **Collect results** in the same DataFrame format shown above

Example integration:
```python
# For each configuration
sessions = session_gen.generate_sessions(
    n_sessions=n_sessions,
    priority_ratio=priority_ratio,
    charging_window_hours=window_hours
)

# Convert to ACN-Sim events (use your existing conversion logic)
events = convert_sessions_to_events(sessions)

# Run simulation with each algorithm
for alg_name, scheduler in sch.items():
    sim = Simulator(cn, scheduler, events, start, period=PERIOD)
    sim.run()
    # Extract metrics...
```